In [1]:
import random
from IPython.display import display, Latex, Markdown
import galois
import numpy as np



# Baby Kyber https://cryptopedia.dev/posts/kyber/


## Parameter

Alle Parameter können frei angepasst werden.
Die Standardwerte sollten zu übersichtlichkeit klein gehalten werden.


In [2]:
randomParameter = False

# ===== Parameter =====
q = 17              # Modulus für Koeffizienten
n = 4               # Grad: Polynomring Z_q[x] / (x^n + 1)
k = 2               # Vektorlänge
kleine_faktoren = [16, 0, 1]  # -1 ≡ 16 (mod 17), 0, 1

q_halbe = (q + 1) // 2

print("q =", q)
print("n =", n)
print("k =", k)
print("⌊q/2⌉ =", q_halbe)


q = 17
n = 4
k = 2
⌊q/2⌉ = 9



## Polynom-Arithmetik

Polynome werden als Listen dargestellt:
[a0, a1, a2, a3]  ↔  a0 + a1·x + a2·x² + a3·x³


In [10]:
# Galois Field GF(17)
GF = galois.GF(q)

def reduce_poly(p):
    """Reduziere Polynom modulo (x^n + 1)"""
    try:
        coeffs = list(p.coefficients())
    except:
        try:
            coeffs = list(p)
        except:
            coeffs = [p]
    
    # Konvertiere zu Python ints zur Verarbeitung
    coeffs = [int(c) for c in coeffs]
    
    # Pad mit Nullen auf mindestens n+1
    while len(coeffs) <= n:
        coeffs = [0] + coeffs
    
    # Reduktion: x^n ≡ -1 mod (x^n + 1)
    # Also x^(n+k) ≡ -x^k = (q - 1)*x^k in GF(17) = 16*x^k
    while len(coeffs) > n:
        # Extrahiere Koeffizienten ab Index n und reduziere
        for i in range(len(coeffs) - n):
            coeffs[i] = (coeffs[i] + 16 * coeffs[n + i]) % q
        # Schneide ab auf Länge n
        coeffs = coeffs[:n]
    
    # Konvertiere zurück zu GF(q) Elementen
    coeffs = [GF(c) for c in coeffs]
    
    return galois.Poly(coeffs, field=GF)

def zufalls_polynom(klein=True):
    """Generiere zufälliges Polynom mit Koeffizienten in GF(q)"""
    if klein:
        coeffs = [GF(random.choice(kleine_faktoren)) for _ in range(n)]
    else:
        coeffs = [GF(random.randint(0, q-1)) for _ in range(n)]
    return galois.Poly(coeffs, field=GF)

def poly_str(p):
    """Formatiert ein Polynom als lesbare String mit x^i Notation"""
    if isinstance(p, list):
        # Falls es noch eine Liste ist
        terms = []
        for i, coeff in enumerate(p):
            coeff = int(coeff) % q
            if coeff == 0:
                continue
            if i == 0:
                terms.append(str(coeff))
            elif i == 1:
                if coeff == 1:
                    terms.append("x")
                else:
                    terms.append(f"{coeff}·x")
            else:
                if coeff == 1:
                    terms.append(f"x^{i}")
                else:
                    terms.append(f"{coeff}·x^{i}")
        if not terms:
            return "0"
        return " + ".join(terms)
    else:
        # Galois Poly Objekt - coefficients() ist eine Methode
        try:
            coeffs = list(p.coefficients())
        except:
            try:
                coeffs = list(p)
            except:
                coeffs = [p]
        terms = []
        for i in range(len(coeffs)):
            coeff = int(coeffs[-(i+1)])
            if coeff == 0:
                continue
            if i == 0:
                terms.append(str(coeff))
            elif i == 1:
                if coeff == 1:
                    terms.append("x")
                else:
                    terms.append(f"{coeff}x")
            else:
                if coeff == 1:
                    terms.append(f"x^{i}")
                else:
                    terms.append(f"{coeff}x^{i}")
        if not terms:
            return "0"            
        return " + ".join(reversed(terms))

def matrix_latex(M):
    """Formatiert eine Matrix als LaTeX und gibt Latex-Objekt zurück"""
    rows = []
    for row in M:
        row_str = " & ".join([poly_str(p) for p in row])
        rows.append(row_str)
    matrix_str = " \\\\ ".join(rows)
    latex_str = f"\\begin{{pmatrix}} {matrix_str} \\end{{pmatrix}}"
    return Latex(latex_str)

def vector_latex(v):
    """Formatiert einen Vektor als LaTeX und gibt Latex-Objekt zurück"""
    row_str = " \\\\ ".join([poly_str(p) for p in v])
    latex_str = f"\\begin{{pmatrix}} {row_str} \\end{{pmatrix}}"
    return Latex(latex_str)

def poly_latex(v):
    """Formatiert ein Polynom als zentriertes LaTeX-Objekt"""
    latex_str = r"\[ " + str(v) + r" \]"
    return Latex(latex_str)

def mat_vec_mul(A, v):
    """Matrix-Vektor Multiplikation mit Polynom-Reduktion modulo (x^n + 1)"""
    out = []
    for row in A:
        acc_poly = galois.Poly([GF(0)], field=GF)
        for a, b in zip(row, v):
            # Multipliziere Polynome und reduziere sofort
            prod = a * b
            prod = reduce_poly(prod)
            acc_poly = acc_poly + prod
            acc_poly = reduce_poly(acc_poly)
        out.append(acc_poly)
    return out



## Schlüsselgenerierung durch Alice


In [15]:
# Privater Schlüssel s
if randomParameter:
    s = [zufalls_polynom(klein=True) for _ in range(k)]
else:
    # s=(−x3−x2+x,−x3−x)
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3] = [const, x, x², x³]
    s = [galois.Poly([16, 16, 1, 0], field=GF), galois.Poly([16, 0, 16, 0], field=GF)]

# Fehlervektor e
if randomParameter:
    e = [zufalls_polynom(klein=True) for _ in range(k)]
else:
    # e=(x2,x2−x)
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3]
    e = [galois.Poly([0, 1, 0, 0], field=GF), galois.Poly([0, 1, 16, 0], field=GF)]

# Öffentliche Matrix A
if randomParameter:
    A = [[zufalls_polynom(klein=False) for _ in range(k)] for _ in range(k)]
else:
    # A = [[6x³+16x²+16x+11, 9x³+4x²+6x+3],
    #      [5x³+3x²+10x+1,  6x³+x²+9x+15]]
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3] = [const, x, x², x³]
    A = [[galois.Poly([6, 16, 16, 11], field=GF), galois.Poly([9, 4, 5, 3], field=GF)],
         [galois.Poly([5, 3, 10, 1], field=GF), galois.Poly([6, 1, 9, 15], field=GF)]]
    
display(Markdown("**Zufällig generierter Privater Schlüssel s:**"))
display(vector_latex(s))

display(Markdown("**Zufällig generierter Fehlervektor e:**"))
display(vector_latex(e))

display(Markdown("**Zufällig generierte Öffentliche Matrix A:**"))
display(matrix_latex(A))

# t = A*s + e mit Reduktion nach jeder Operation
t = []
for i in range(k):
    poly_sum = galois.Poly([GF(0)], field=GF)
    for j in range(k):
        prod = A[i][j] * s[j]
        prod = reduce_poly(prod)
        poly_sum = poly_sum + prod
        poly_sum = reduce_poly(poly_sum)
    t.append(reduce_poly(poly_sum + e[i]))

display(Markdown(r"**Öffentlicher Vektor t**"))
display(Markdown(r"\\[ \mathbf{t} = A\mathbf{s} + \mathbf{e} \\]"))
display(vector_latex(t))


**Zufällig generierter Privater Schlüssel s:**

<IPython.core.display.Latex object>

**Zufällig generierter Fehlervektor e:**

<IPython.core.display.Latex object>

**Zufällig generierte Öffentliche Matrix A:**

<IPython.core.display.Latex object>

**Öffentlicher Vektor t**

\\[ \mathbf{t} = A\mathbf{s} + \mathbf{e} \\]

<IPython.core.display.Latex object>


## Verschlüsselung durch Bob


In [5]:
# Zufälliger Vektor r und Fehler
if randomParameter:
    r = [zufalls_polynom(klein=True) for _ in range(k)]
else: 
    # r = (-x³ + x², x³ + x² - 1)
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3]
    r = [galois.Poly([0, 0, 1, 16], field=GF), galois.Poly([16, 0, 1, 1], field=GF)]
if randomParameter:
    e1 = [zufalls_polynom(klein=True) for _ in range(k)]
else:
    # e1 = (x² + x, x²)
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3]
    e1 = [galois.Poly([0, 1, 1, 0], field=GF), galois.Poly([0, 0, 1, 0], field=GF)]
if randomParameter:
    e2 = zufalls_polynom(klein=True)
else:
    # e2 = -x³ - x² = 16x³ + 16x²
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3]
    e2 = galois.Poly([0, 0, 16, 16], field=GF)
    
display(Markdown("**Zufallsvektor r:**"))
display(vector_latex(r))

display(Markdown("**Zufällig generierter Fehler e1:**"))
display(vector_latex(e1))

display(Markdown("**Zufällig generierter Fehler e2**:"))
display(poly_str(e2))

# Nachricht (zufällige 4 Bit)
if randomParameter:
    m_bin = [random.randint(0,1) for _ in range(n)]
else:
    # m_bin entspricht 11 in binary = 1011, als Polynom x³ + x + 1
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3]
    m_bin = [1, 1, 0, 1]
m_coeffs = [GF(q_halbe * x) for x in m_bin]
m = galois.Poly(m_coeffs, field=GF)

display(Markdown("**Binäre Nachricht m_bin:**"))
display(poly_str(m_bin))
display(Markdown("**Skalierte Nachricht m:**"))
display(poly_str(m))

# u = A^T r + e1 mit Reduktion
u = []
for j in range(k):
    poly_sum = galois.Poly([GF(0)], field=GF)
    for i in range(k):
        prod = A[i][j] * r[i]
        prod = reduce_poly(prod)
        poly_sum = poly_sum + prod
        poly_sum = reduce_poly(poly_sum)
    u.append(reduce_poly(poly_sum + e1[j]))

# v = t^T r + e2 + m mit Reduktion
v = galois.Poly([GF(0)], field=GF)
for i in range(k):
    prod = t[i] * r[i]
    prod = reduce_poly(prod)
    v = v + prod
    v = reduce_poly(v)
v = reduce_poly(v + e2 + m)

display(Markdown("**Ciphertext u:**"))
display(Markdown("$\\mathbf{u} = \\mathbf{A}^T \\mathbf{r} + \\mathbf{e}_1$"))
display(vector_latex(u))

display(Markdown("**Ciphertext v**:"))
display(Markdown("$v = \\mathbf{t}^T \\mathbf{r} + e_2 + m$"))
display(poly_str(v))


**Zufallsvektor r:**

<IPython.core.display.Latex object>

**Zufällig generierter Fehler e1:**

<IPython.core.display.Latex object>

**Zufällig generierter Fehler e2**:

...

$\mathbf{u} = \mathbf{A}^T \mathbf{r} + \mathbf{e}_1$

<IPython.core.display.Latex object>

**Ciphertext v**:

$v = \mathbf{t}^T \mathbf{r} + e_2 + m$

'13 + 8x + 12x^2 + 14x^3'


## Entschlüsselung durch Alice


In [6]:
# m_noisy = v - s^T u mit Reduktion
m_noisy = galois.Poly([GF(0)], field=GF)
for i in range(k):
    prod = s[i] * u[i]
    prod = reduce_poly(prod)
    m_noisy = m_noisy + prod
    m_noisy = reduce_poly(m_noisy)
m_noisy = reduce_poly(v - m_noisy)

display(Markdown("**Rauschbehaftete Nachricht m~:**"))
display(Markdown("$m_{\\text{noisy}} = v - \\mathbf{s}^T \\mathbf{u}$"))
display(poly_str(m_noisy))

# Rundung
try:
    m_noisy_coeffs = list(m_noisy.coefficients())
except:
    try:
        m_noisy_coeffs = list(m_noisy)
    except:
        m_noisy_coeffs = [m_noisy]
    
# Pad mit Nullen falls nötig
while len(m_noisy_coeffs) < n:
    m_noisy_coeffs = [GF(0)] + m_noisy_coeffs

m_rec = []
for coeff in reversed(m_noisy_coeffs[-n:]):
    c = int(coeff)
    if abs(c - q_halbe) < abs(c):
        m_rec.append(1)
    else:
        m_rec.append(0)

display(Markdown("**Wiederhergestellte Bits (nach Rundung):**"))
display(poly_str(m_rec))
display(Markdown("**Originale Bits:**"))
display(poly_str(m_bin))

# Vergleich
if m_rec == m_bin:
    display(Markdown("✓ **Erfolgreich dekodiert!**"))
else:
    display(Markdown("✗ **Dekodierung fehlgeschlagen**"))


**Rauschbehaftete Nachricht m~:**

$m_{\text{noisy}} = v - \mathbf{s}^T \mathbf{u}$

'11 + 11x + 10x^2 + 15x^3'

**Wiederhergestellte Bits (nach Rundung):**

'1 + x + x^2 + x^3'

**Originale Bits:**

'1 + x + x^3'

✗ **Dekodierung fehlgeschlagen**


## Zusammenfassung

- Alle Werte zufällig erzeugt
- Jeder Zwischenschritt ausgegeben
- Klassische Baby-Kyber-Darstellung
- Ideal zum Nachrechnen auf Papier

So wurde Kryptographie traditionell erklärt: klein, offen, nachvollziehbar.
